<a href="https://colab.research.google.com/github/Salva13s/HybridModel/blob/main/NuevoDino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

folder_path = '/content/drive/MyDrive/1 Tesis/Data/PersonDataset/test/Real'
images = os.listdir(folder_path)
#print(images)

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .

/content
fatal: destination path 'GroundingDINO' already exists and is not an empty directory.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino-py 0.4.0 requires supervision==0.6.0, but you have supervision 0.25.0 which is incompatible.


In [ ]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
mkdir: cannot create directory ‘/content/weights’: File exists
/content/weights


In [ ]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [ ]:
!pip install groundingdino-py

#Prediccion y guardado de coordenadas y boundingbox


In [ ]:
import os
import numpy as np
import tensorflow as tf
from groundingdino.util.inference import load_model, load_image, predict, annotate
import supervision as sv
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/1 Tesis/Data/TreeDataR/Val/Reales'
detections_path = '/content/drive/MyDrive/1 Tesis/Data/DetectTreeVal'
boxes_path = '/content/drive/MyDrive/1 Tesis/Data/BoxesTreeVal'

os.makedirs(detections_path, exist_ok=True)
os.makedirs(boxes_path, exist_ok=True)

%cd /content/groundingdino


model = load_model(CONFIG_PATH, WEIGHTS_PATH)

image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]

processed_images = []

for image_name in image_files:
    image_path = os.path.join(folder_path, image_name)
    image_source, image = load_image(image_path)

    TEXT_PROMPT = "Tree"
    BOX_TRESHOLD = 0.30
    TEXT_TRESHOLD = 1

    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )

    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

    npy_image_path = os.path.join(detections_path, f"{os.path.splitext(image_name)[0]}_detecciones.npy")
    np.save(npy_image_path, annotated_frame)

    boxes_file_path = os.path.join(boxes_path, f"{os.path.splitext(image_name)[0]}_boxes.npy")
    np.save(boxes_file_path, boxes.numpy() if isinstance(boxes, tf.Tensor) else boxes)

    processed_images.append(image_name)

print(f"Procesamiento completado. Total de imágenes procesadas: {len(processed_images)}")
print("Imágenes procesadas:")
for img in processed_images:
    print(f"- {img}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/groundingdino'
/content/weights
final text_encoder_type: bert-base-uncased
Procesamiento completado. Total de imágenes procesadas: 52
Imágenes procesadas:
- ck2mc4a7nss4b0748bcn8tobg.jpg
- ck2jba6env4w20748456ta3ws.jpg
- ck2si6p9r0svl0794nqd113r1.jpg
- ck2goa3nkm4p107947m7zs86p.jpg
- ck2ybj5cgn3790757hmloxpkp.jpg
- ck4cd67gnyltx0921b4wa9ka3.jpg
- ck2t118g07oc10944lhdj09vc.jpg
- ck2gl4ztbbsw40725l6sha0hh.jpg
- ck2oj2qcb1ewu0757h2ucou0l.jpg
- ck2oiwrlc1bck07570zwbjxp2.jpg
- ck2omywe5ywj50944fjtwtqbv.jpg
- ck2kvgnyz2igc0725j0lposxc.jpg
- ck2szq5voby9x0757fl3wjgin.jpg
- ck4ced9qi01bt0706rjsm99ft.jpg
- ck2oieg0210qq0757okd1grmd.jpg
- ck2jfqs74xjw70794j78lxs00.jpg
- ck2uu18aqjydt0748mo4nlkcr.jpg
- ck4cg5x6b1bov07066c2y94rz.jpg
- ck2yb6xczdw540794fotv39pt.jpg
- ck2oi9y07wddg0944b1xu8r7f.jpg
- ck2lte0nfngwt0794cxyfsqb8.j

#Generacion puntos para pseudo mapa de densidad


#Crear Pseudo Mapas de densidad


In [ ]:
import os
import cv2
import numpy as np

#detections_path = '/content/drive/MyDrive/1 Tesis/Data/DetectTreeTest'
#boxes_path = '/content/drive/MyDrive/1 Tesis/Data/BoxesTreeTest'

boxes_path = '/content/drive/MyDrive/1 Tesis/Data/BoxesTreeVal'

output_folder = '/content/drive/MyDrive/1 Tesis/Data/TreeDataR/Val/PseudoDense'

os.makedirs(output_folder, exist_ok=True)

boxes_files = [f for f in os.listdir(boxes_path) if f.endswith('.npy')]

for boxes_file in boxes_files:
    boxes_file_path = os.path.join(boxes_path, boxes_file)
    boxes = np.load(boxes_file_path)

    black_image = np.zeros((640, 640, 3), dtype=np.uint8)
    for box in boxes:
        x_center = int(box[0] * 640)
        y_center = int(box[1] * 640)

        cv2.circle(black_image, (x_center, y_center), radius=5, color=(255, 255, 255), thickness=-1)

    imagen_filtrada = cv2.GaussianBlur(black_image, (5, 5), 0)

    imagen_normalizada = cv2.normalize(imagen_filtrada, None, 0, 255, cv2.NORM_MINMAX)

    output_filename = boxes_file.replace('_boxes.npy', '.png')
    output_image_path = os.path.join(output_folder, output_filename)

    cv2.imwrite(output_image_path, imagen_normalizada)

    print(f"Imagen filtrada y normalizada guardada en: {output_image_path}")
